**Load python libraries**

In [1]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import os.path
import xarray as xr
import pandas as pd
import numpy as np
import requests
import netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from IPython.display import Image, display
from dask.distributed import Client, progress
from glob import glob
import colorsys
from matplotlib.colors import LinearSegmentedColormap # Linear interpolation for color maps
import pylab as pl
from shapely.geometry import Point
from descartes import PolygonPatch
import wrf

from modules.GOESR_functions import goes_lat_lon_reproj
from modules.concave_hull import alpha_shape, plot_polygon

#### Import CFSv2 0* Isotherm Data

In [2]:
def preprocess(ds):
    '''keep only the selected lats and lons'''
    return ds.sel(latitude=slice(10,50), longitude=slice(210,250))

filename_pattern =  '../data/CFSv2/netcdf_CFSv2_vertical_levels/201*'
ds = xr.open_mfdataset(filename_pattern, engine='netcdf4', preprocess=preprocess)
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
ds = ds.sel(time=slice('2019-03-04-18', '2019-03-06-18'))
ds['temp'] = ds.temp - 273.15 ## convert to degrees Celsius
zero_isotherm = ds.hgt_0Cisotherm

ds size in GB 0.17



Check for 0* crossings in the temperature profile between 1000 and 200 hPa.
If single zero crossing, altitude is taken as the freezing level.
Special case 1: No zero crossing (T < 0degC throughout column)
Flag freezing level as missing.
Special case 2: Multiple zero crossing due to temperature inversion
Flag all locations, store only lowest Z0 value

In [4]:
ds.hgtprs

<xarray.DataArray 'hgtprs' (time: 9, p: 37, latitude: 81, longitude: 81)>
dask.array<shape=(9, 37, 81, 81), dtype=float32, chunksize=(1, 37, 81, 81)>
Coordinates:
  * latitude   (latitude) float64 10.0 10.5 11.0 11.5 ... 48.5 49.0 49.5 50.0
  * longitude  (longitude) float64 210.0 210.5 211.0 211.5 ... 249.0 249.5 250.0
  * p          (p) float32 1000.0 975.0 950.0 925.0 900.0 ... 5.0 3.0 2.0 1.0
  * time       (time) datetime64[ns] 2019-03-04T18:00:00 ... 2019-03-06T18:00:00
Attributes:
    short_name:  hgtprs
    long_name:   Geopotential Height
    level:       ��M
    units:       m

In [ ]:
ht = ds.hgtprs
t = ds.temp

interp_results = wrf.interplevel(ht, t, 0.0, missing=np.nan)
interp_results

In [ ]:
fig = plt.figure(figsize=(13., 8.0))
fig.dpi = 600
fname = './figures/zero_isotherm_test1'
fmt = 'png'

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

subtitles = ['(a) 4 March 2019 18 UTC', 
             '(b) 5 March 2019 00 UTC', '(c) 5 March 2019 06 UTC', '(d) 5 March 2019 12 UTC',
             '(e) 5 March 2019 18 UTC',
            '(f) 6 March 2019 00 UTC', '(g) 6 March 2019 06 UTC', '(h) 6 March 2019 12 UTC',
             '(i) 6 March 2019 18 UTC']
# ext = (-138., -118.0, 24.0, 44.0)
ext = [-115.0, -140.0, 25.0, 40.0]
clevs = np.arange(0, 16000, 1000)
clevs_thick = np.arange(0, 13000, 1000)
clevs_fl = np.arange(0, 5250, 250)

for i in np.arange(len(zero_isotherm.values)):
    ax = plt.subplot(3, 3, i+1, projection=mapcrs)
    ax.set_extent(ext, crs=mapcrs)
    cf = ax.contourf(ds.longitude, ds.latitude, zero_isotherm.values[i], transform=datacrs,
                       cmap='jet', levels=clevs_fl)
    
    
    ax.coastlines(linewidths=1.0, resolution='10m')
    ax.add_feature(states_provinces, edgecolor='k')
    ax.add_feature(cfeature.BORDERS)
    ax.set_title(subtitles[i], fontsize=10)
    ## Add in meridian and parallels
    gl = ax.gridlines(crs=mapcrs, draw_labels=True,
                  linewidth=.5, color='black', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator(np.arange(-140., -110., 4))
    gl.ylocator = mticker.FixedLocator(np.arange(24, 44, 2))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 7}
    gl.ylabel_style = {'size': 7}

# # # add colorbar [left, bottom, width, height]
ax2 = fig.add_axes([0.9, 0.13, 0.03, 0.75])
cbar = fig.colorbar(cf, cax=ax2, drawedges=True, 
                    orientation='vertical', extendfrac='auto')
cbar.ax.tick_params(labelsize=8)
cbar.set_label('$\mathrm{0 \degree \: isotherm \: (m)}$', fontsize=12)

plt.subplots_adjust(hspace=0.2, wspace=0.003)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()


plotFile = fname + '.png'
print(plotFile)
display(Image(plotFile))

In [ ]:
fig = plt.figure(figsize=(13., 8.0))
fig.dpi = 600
fname = './figures/zero_isotherm_test2'
fmt = 'png'

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

subtitles = ['(a) 4 March 2019 18 UTC', 
             '(b) 5 March 2019 00 UTC', '(c) 5 March 2019 06 UTC', '(d) 5 March 2019 12 UTC',
             '(e) 5 March 2019 18 UTC',
            '(f) 6 March 2019 00 UTC', '(g) 6 March 2019 06 UTC', '(h) 6 March 2019 12 UTC',
             '(i) 6 March 2019 18 UTC']
# ext = (-138., -118.0, 24.0, 44.0)
ext = [-115.0, -140.0, 25.0, 40.0]
clevs = np.arange(0, 16000, 1000)
clevs_thick = np.arange(0, 13000, 1000)
clevs_fl = np.arange(0, 5250, 250)

for i in np.arange(len(zero_isotherm.values)):
    ax = plt.subplot(3, 3, i+1, projection=mapcrs)
    ax.set_extent(ext, crs=mapcrs)
    cf = ax.contourf(ds.longitude, ds.latitude, interp_results.values[i], transform=datacrs,
                       cmap='jet', levels=clevs_fl)
    
    
    ax.coastlines(linewidths=1.0, resolution='10m')
    ax.add_feature(states_provinces, edgecolor='k')
    ax.add_feature(cfeature.BORDERS)
    ax.set_title(subtitles[i], fontsize=10)
    ## Add in meridian and parallels
    gl = ax.gridlines(crs=mapcrs, draw_labels=True,
                  linewidth=.5, color='black', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator(np.arange(-140., -110., 4))
    gl.ylocator = mticker.FixedLocator(np.arange(24, 44, 2))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 7}
    gl.ylabel_style = {'size': 7}

# # # add colorbar [left, bottom, width, height]
ax2 = fig.add_axes([0.9, 0.13, 0.03, 0.75])
cbar = fig.colorbar(cf, cax=ax2, drawedges=True, 
                    orientation='vertical', extendfrac='auto')
cbar.ax.tick_params(labelsize=8)
cbar.set_label('$\mathrm{0 \degree \: isotherm \: (m)}$', fontsize=12)

plt.subplots_adjust(hspace=0.2, wspace=0.003)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()


plotFile = fname + '.png'
print(plotFile)
display(Image(plotFile))

In [ ]:
fig = plt.figure(figsize=(13., 8.0))
fig.dpi = 600
fname = './figures/zero_isotherm_testbias'
fmt = 'png'

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

subtitles = ['(a) 4 March 2019 18 UTC', 
             '(b) 5 March 2019 00 UTC', '(c) 5 March 2019 06 UTC', '(d) 5 March 2019 12 UTC',
             '(e) 5 March 2019 18 UTC',
            '(f) 6 March 2019 00 UTC', '(g) 6 March 2019 06 UTC', '(h) 6 March 2019 12 UTC',
             '(i) 6 March 2019 18 UTC']
# ext = (-138., -118.0, 24.0, 44.0)
ext = [-115.0, -140.0, 25.0, 40.0]
clevs = np.arange(-1200, 1300, 100)

for i in np.arange(len(zero_isotherm.values)):
    ax = plt.subplot(3, 3, i+1, projection=mapcrs)
    ax.set_extent(ext, crs=mapcrs)
    result = interp_results.values[i] - zero_isotherm.values[i]
    cf = ax.contourf(ds.longitude, ds.latitude, result, transform=datacrs,
                       cmap='coolwarm', levels=clevs)
    
    
    ax.coastlines(linewidths=1.0, resolution='10m')
    ax.add_feature(states_provinces, edgecolor='k')
    ax.add_feature(cfeature.BORDERS)
    ax.set_title(subtitles[i], fontsize=10)
    ## Add in meridian and parallels
    gl = ax.gridlines(crs=mapcrs, draw_labels=True,
                  linewidth=.5, color='black', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator(np.arange(-140., -110., 4))
    gl.ylocator = mticker.FixedLocator(np.arange(24, 44, 2))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 7}
    gl.ylabel_style = {'size': 7}

# # # add colorbar [left, bottom, width, height]
ax2 = fig.add_axes([0.9, 0.13, 0.03, 0.75])
cbar = fig.colorbar(cf, cax=ax2, drawedges=True, 
                    orientation='vertical', extendfrac='auto')
cbar.ax.tick_params(labelsize=8)
cbar.set_label('$\mathrm{0 \degree \: isotherm \: (m)}$', fontsize=12)

plt.subplots_adjust(hspace=0.2, wspace=0.003)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()


plotFile = fname + '.png'
print(plotFile)
display(Image(plotFile))